In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2OGeneralizedLinearEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.automl import H2OAutoML
pd.options.mode.chained_assignment = None 
import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import statsmodels 
import fbprophet 
from pyramid.arima import auto_arima

In [5]:
#h2o.connect(ip='aamonster')
h2o.connect()

Connecting to H2O server at http://localhost:54321... successful.


H2O cluster uptime:,32 secs
H2O cluster timezone:,Europe/Belgrade
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.7
H2O cluster version age:,4 months and 5 days !!!
H2O cluster name:,H2O_from_python_asologuba_aekww7
H2O cluster total nodes:,1
H2O cluster free memory:,3.517 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


<H2OConnection to http://localhost:54321, no session>

In [8]:
data = pd.read_csv('data_prep.csv')
train_test = data[data.id == -1]
valid = data[data.id > -1]

stores = data.store.unique()
products = data.item.unique()

train_test_h2o = h2o.H2OFrame(train_test)
valid_h2o = h2o.H2OFrame(valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


AVERAGE VALUE FROM LAST 5 YEARS

In [14]:
results_slim = valid[['MONTH', 'DAY', 'id', 'store', 'item']]
data_slim = data[['MONTH', 'DAY', 'store', 'item', 'sales', 'date']]
resutls_s = results_slim.merge(data_slim, how='left', on = ['MONTH', 'DAY', 'store', 'item'])

results_avg = resutls_s.groupby(['id'])['sales'].mean()
results_avg = DataFrame(results_avg)
results_avg.reset_index(inplace = True)

VALUE FROM LAST PERIOD

In [15]:
results_slim = valid[['MONTH', 'DAY', 'id', 'store', 'item']]
data_slim = data[data.YEAR == 2017]
data_slim = data_slim[['MONTH', 'DAY', 'store', 'item', 'sales', 'date']]

results_last = results_slim.merge(data_slim, how='left', on = ['MONTH', 'DAY', 'store', 'item'])
results_last = results_last[['id', 'sales']]
results_last.sort_values('id', inplace = True)
results_last.reset_index(drop=True, inplace = True)

## GLM H2O

In [72]:
model = H2OGeneralizedLinearEstimator(family='gaussian')

results = DataFrame(columns=['id', 'sales'])

for s in stores:
    for p in products:
        train_test_sp_h2o = train_test_h2o[train_test_h2o['store'] == float(s)]
        train_test_sp_h2o = train_test_sp_h2o[train_test_sp_h2o['item'] == float(p)]
        valid_sp_h2o = valid_h2o[valid_h2o['store'] == float(s)]
        valid_sp_h2o = valid_sp_h2o[valid_sp_h2o['item'] == float(p)]
        
        model.train(x = train_test_sp_h2o.columns[4:-1],
                    y = train_test_sp_h2o.columns[-1],
                    training_frame = train_test_sp_h2o,
                    max_runtime_secs = 100)
        prediction = model.predict(valid_sp_h2o[4:-1])
        prediction_df = h2o.as_list(prediction)
        
        results_sp = h2o.as_list(valid_sp_h2o['id'])
        results_sp['sales'] = prediction_df
        results = results.append(results_sp)
results_glm = results

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |█

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |█

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |█

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |█

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm Model Build progress: |█

## GBM H2O separatly for stores and products, then combining using average

In [18]:
model = H2OGradientBoostingEstimator(ntrees=10, 
                        max_depth = 5,
                        stopping_metric = 'mae',
                        nfolds = 5) 

results = DataFrame(columns=['id', 'sales'])

for s in stores:
    train_test_s = train_test_h2o[train_test_h2o['store'] == float(s)]
    valid_s = valid_h2o[valid_h2o['store'] == float(s)]

    model.train(x = train_test_s.columns[2:-1],
                y = train_test_s.columns[-1],
                training_frame = train_test_s,
                max_runtime_secs = 10)
    prediction = model.predict(valid_s[2:-1])
    prediction_df = h2o.as_list(prediction)

    results_sp = h2o.as_list(valid_s['id'])
    results_sp['sales'] = prediction_df
    results = results.append(results_sp)

results_s_gbm = results   

results = DataFrame(columns=['id', 'sales'])

for p in products:
    train_test_p = train_test_h2o[train_test_h2o['item'] == float(p)]
    valid_p = valid_h2o[valid_h2o['item'] == float(p)]

    model.train(x = train_test_p.columns[2:-1],
                y = train_test_p.columns[-1],
                training_frame = train_test_p,
                max_runtime_secs = 10)
    prediction = model.predict(valid_p[2:-1])
    prediction_df = h2o.as_list(prediction)

    results_sp = h2o.as_list(valid_p['id'])
    results_sp['sales'] = prediction_df
    results = results.append(results_sp)

results_p_gbm = results

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |█

gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |██

# fbprophet

In [80]:
results = DataFrame(columns=['id', 'sales'])

for s in stores:
    for p in products:
        print('store id  '+str(s))
        print('product id '+str(p))
        valid_sp = valid.loc[valid.store == s]
        valid_sp = valid_sp.loc[valid_sp.item == p]
        valid_sp.reset_index(inplace = True, drop = True)
        
        train_test_sp = train_test.loc[train_test.store == s]
        train_test_sp = train_test_sp.loc[train_test_sp.item == p]
        train_test_sp.reset_index(inplace = True, drop = True)
        tt = train_test_sp[['date', 'sales']]
        tt.columns = ['ds', 'y']
        gm_prophet = fbprophet.Prophet()
        gm_prophet.fit(tt)
        gm_forecast = gm_prophet.make_future_dataframe(periods = 90, freq='D')
        gm_forecast = gm_prophet.predict(gm_forecast)

        results_sp = DataFrame(valid_sp.id)
        gm_results = gm_forecast[gm_forecast.ds >= '2018-01-01'].yhat
        gm_results.reset_index(inplace = True, drop = True)
        results_sp['sales'] = gm_results
        results = results.append(results_sp)
results_fb = results

sklep 1
produkt 1
sklep 1
produkt 2
sklep 1
produkt 3
sklep 1
produkt 4
sklep 1
produkt 5
sklep 1
produkt 6
sklep 1
produkt 7
sklep 1
produkt 8
sklep 1
produkt 9
sklep 1
produkt 10
sklep 1
produkt 11
sklep 1
produkt 12
sklep 1
produkt 13
sklep 1
produkt 14
sklep 1
produkt 15
sklep 1
produkt 16
sklep 1
produkt 17
sklep 1
produkt 18
sklep 1
produkt 19
sklep 1
produkt 20
sklep 1
produkt 21
sklep 1
produkt 22
sklep 1
produkt 23
sklep 1
produkt 24
sklep 1
produkt 25
sklep 1
produkt 26
sklep 1
produkt 27
sklep 1
produkt 28
sklep 1
produkt 29
sklep 1
produkt 30
sklep 1
produkt 31
sklep 1
produkt 32
sklep 1
produkt 33
sklep 1
produkt 34
sklep 1
produkt 35
sklep 1
produkt 36
sklep 1
produkt 37
sklep 1
produkt 38
sklep 1
produkt 39
sklep 1
produkt 40
sklep 1
produkt 41
sklep 1
produkt 42
sklep 1
produkt 43
sklep 1
produkt 44
sklep 1
produkt 45
sklep 1
produkt 46
sklep 1
produkt 47
sklep 1
produkt 48
sklep 1
produkt 49
sklep 1
produkt 50
sklep 2
produkt 1
sklep 2
produkt 2
sklep 2
produkt 3
sklep

Joining all results into the average

In [271]:
results = DataFrame((results_glm.sales + results_fb.sales + results_last.sales + results_srednia.sales)/4)
results.reset_index(inplace = True)
results.columns = ['id', 'sales']
results.to_csv('results.csv', index=False)